# Chatbot

Utilize the chat format to have extended conversations with the chatbots personalized or specialized to your needs.

In [ ]:
from utils import get_completion_from_messages
from utils import collect_messages


In [ ]:
messages = [
    {'role': 'system', 'content': 'You are an assistant that speaks like Harry Potter.'},
    {'role': 'user', 'content': 'tell me a joke'},
    {'role': 'assistant', 'content': 'Why did the chicken cross the road'},
    {'role': 'user', 'content': 'I don\'t know'}
]


In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)


In [ ]:
messages = [
    {'role': 'system', 'content': 'You are friendly chatbot.'},
    {'role': 'user', 'content': 'Hi, my name is Bob'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)


The model does not remember previous interactions.

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are friendly chatbot.'},
    {'role': 'user', 'content': 'Yes,  can you remind me, What is my name?'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)


Each interaction with the language model is a standalone interaction, i.e. the model does not remember the previous interactions. This is a limitation of the current implementation of the chatbot.

If you want to have a conversation with the chatbot, you need to provide the context of the conversation in each interaction, i.e remember from the previous interaction.

In [ ]:
messages = [
    {'role': 'system', 'content': 'You are friendly chatbot.'},
    {'role': 'user', 'content': 'Hi, my name is Bob'},
    {'role': 'assistant', 'content': "Hi Bob! It's nice to meet you. Is there anything I can help you with today?"},
    {'role': 'user', 'content': 'Yes, you can remind me, What is my name?'}]
response = get_completion_from_messages(messages, temperature=1)
print(response)


## MomoBot

We can automate the collection of user prompts and assistant reponses to build a chatbot.

The MomoBot will take orders at a pizza restaurant.

In [ ]:
import panel as pn
pn.extension()

panels = []  # collect display


In [ ]:
# function for chatbot to collect messages
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''

    context.append(
        {
            'role': 'user',
            'content': f"{prompt}"
        }
    )
    response = get_completion_from_messages(context)

    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row(
            'User:', pn.pane.Markdown(prompt, width=600)
        )
    )

    panels.append(
        pn.Row(
            'Assistant:', pn.pane.Markdown(response, width=600, style={
                                           'background-color': '#F6F6F6'})
        )
    )

    return pn.Column(*panels)


In [ ]:
panels = []  # collect display

context = [
    {
        'role': 'system',
        'content': '''
You are MomoBot, an automated service to collect orders for a momo restaurant. \
You first greet the customer, then collect the order, and then ask if it's a pickup or dine-in. \
You wait to collect the entire order, then summarize it and \
check for a final time if the customer wants to add anything else. \
Now give the total amount and ask for payment. \
The amount is in Rs.\
Finally, you collect the payment.\
Make sure to clarify all options, types and drinks, \
to uniquely identify the item from the menu.\
Drinks are arranged in 1litre, 1.5litre , 2litre bottles.\
You respond in a short, very conversational friendly style. \
If the customer asks for something that is not on the menu, \
you respond with "Sorry, we don't have that." \
The menu includes \
Buff Momo 200
Chicken Momo 200
Veg Momo 160
Pork Momo 200
Paneer Momo 210 \
Types: \
Steam 0
Kothey 40
Fried 20 \
C 50 \
Jhol 30 \
Drinks: \
Coke 90, 180, 250\
Fanta 90, 180, 250 \
Sprite 90, 180, 250 \
Bottled water 69
    '''
    }
]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(
    collect_messages,
    button_conversation
)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)


Display the dashboard of the chatbot.


In [ ]:
dashboard


Now, create a json file for the orders.

In [ ]:
messages = context.copy()
messages.append(
    {
        'role': 'system',
        'content': 'Create a json summary of the previous food order.\
            Itemize the price for each item\
            The fields should be 1) Momo, include type 2) List of drinks, include size\
            3) Total price'
    }
)

response = get_completion_from_messages(messages, temperature=0)
print(response)
